## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тестовое множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации документа по категориям, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

###TASK 1

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np

In [3]:
TRAIN_PATH = "/content/drive/MyDrive/news/news_train.txt"
TEST_PATH = "/content/drive/MyDrive/news/news_test.txt"

In [4]:
train_data = pd.read_csv(TRAIN_PATH, sep="\t", header=None, names=['topic', 'headline', 'info'])
test_data = pd.read_csv(TEST_PATH, sep="\t", header=None, names=['topic', 'headline', 'info'])

In [5]:
train_data.head()

,topic,headline,info
0,sport,Овечкин пожертвовал детской хоккейной школе ав...,Нападающий «Вашингтон Кэпиталз» Александр Овеч...
1,culture,Рекордно дорогую статую майя признали подделкой,"Власти Мексики объявили подделкой статую майя,..."
2,science,Samsung представила флагман в защищенном корпусе,Южнокорейская Samsung анонсировала защищенную ...
3,sport,С футболиста «Спартака» сняли четырехматчевую ...,Контрольно-дисциплинарный комитет (КДК) РФС сн...
4,media,Hopes & Fears объединится с The Village,Интернет-издание Hopes & Fears объявило о свое...


In [6]:
!pip install pymorphy2

import pymorphy2
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences

rus_stopwords = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
def text_preprocessing(text):
  punctuation = string.punctuation + '«' + '»' + '—'
  text = "".join(ch if ch not in punctuation else ' ' for ch in text)
  text = "".join([ch if not ch.isdigit() else ' ' for ch in text])
  words = text.split()
  tokens = []
  for word in words:
    word = morph.parse(word)[0].normal_form
    if word not in rus_stopwords:
      tokens.append(word)
  text = " ".join(tokens)
  return text

In [8]:
for i in range(len(train_data)):
  train_data["headline"][i] = text_preprocessing(train_data["headline"][i])
  train_data["info"][i] = text_preprocessing(train_data["info"][i])

In [9]:
train_data.head()

,topic,headline,info
0,sport,овечкин пожертвовать детский хоккейный школа а...,нападать вашингтон кэпиталзти александр овечки...
1,culture,рекордно дорогой статуя майя признать подделка,власть мексика объявить подделка статуя майя п...
2,science,samsung представить флагман защитить корпус,южнокорейский samsung анонсировать защитить ве...
3,sport,футболист спартак снять четырехматчевой дисква...,контрольный дисциплинарный комитет кдк рфс сня...
4,media,hopes fears объединиться the village,интернет издание hopes fears объявить свой сли...


In [10]:
for i in range(len(test_data)):
  test_data["headline"][i] = text_preprocessing(test_data["headline"][i])
  test_data["info"][i] = text_preprocessing(test_data["info"][i])

In [11]:
# train_data = pd.read_csv("/content/drive/MyDrive/news/preprocessed_train.txt")
# test_data = pd.read_csv("/content/drive/MyDrive/news/preprocessed_test.txt")

In [12]:
label_encoder = LabelEncoder()
train_data['target'] = label_encoder.fit_transform(train_data['topic'])
test_data['target'] = label_encoder.transform(test_data['topic'])

In [13]:
# train_data.to_csv( "/content/drive/MyDrive/news/preprocessed_train.txt", index=False, encoding='utf-8-sig')
# test_data.to_csv( "/content/drive/MyDrive/news/preprocessed_test.txt", index=False, encoding='utf-8-sig')

###TASK 2

In [14]:
from gensim.models import Word2Vec

In [15]:
words = [text.split() for text in train_data["info"]]
w2v = Word2Vec(sentences=words, min_count=0)

In [16]:
w2v.wv.most_similar('интернет', topn=10)  

[('соцсеть', 0.8297642469406128),
 ('доступ', 0.8284081816673279),
 ('провайдер', 0.8271694183349609),
 ('онлайн', 0.8270543217658997),
 ('поисковик', 0.826681911945343),
 ('запрос', 0.8212244510650635),
 ('яндекс', 0.8211380243301392),
 ('ресурс', 0.8202977180480957),
 ('реклама', 0.8144828081130981),
 ('контент', 0.8100818395614624)]

###TASK 3

In [17]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
vectorizer = TfidfVectorizer(min_df = 10, max_df = 1., ngram_range = (1,2), max_features = 1000)
train_text = vectorizer.fit_transform(train_data["info"]).toarray()
test_text = vectorizer.transform(test_data["info"]).toarray()
train_labels = np.array(train_data['target'])
test_labels = np.array(test_data['target'])

In [19]:
param_grid = {'C': [1, 10, 100], 'gamma': [0.01, 0.1, 1],'kernel': ['linear', 'sigmoid']}
model = SVC()
clf = GridSearchCV(model, param_grid, cv=2, verbose=3)
clf.fit(train_text, train_labels)

Fitting 2 folds for each of 18 candidates, totalling 36 fits
[CV 1/2] END ....C=1, gamma=0.01, kernel=linear;, score=0.836 total time=  20.1s
[CV 2/2] END ....C=1, gamma=0.01, kernel=linear;, score=0.826 total time=  19.4s
[CV 1/2] END ...C=1, gamma=0.01, kernel=sigmoid;, score=0.514 total time= 1.1min
[CV 2/2] END ...C=1, gamma=0.01, kernel=sigmoid;, score=0.531 total time= 1.1min
[CV 1/2] END .....C=1, gamma=0.1, kernel=linear;, score=0.836 total time=  20.1s
[CV 2/2] END .....C=1, gamma=0.1, kernel=linear;, score=0.826 total time=  20.2s
[CV 1/2] END ....C=1, gamma=0.1, kernel=sigmoid;, score=0.805 total time=  39.4s
[CV 2/2] END ....C=1, gamma=0.1, kernel=sigmoid;, score=0.800 total time=  37.7s
[CV 1/2] END .......C=1, gamma=1, kernel=linear;, score=0.836 total time=  19.4s
[CV 2/2] END .......C=1, gamma=1, kernel=linear;, score=0.826 total time=  19.7s
[CV 1/2] END ......C=1, gamma=1, kernel=sigmoid;, score=0.831 total time=  19.1s
[CV 2/2] END ......C=1, gamma=1, kernel=sigmoid;

GridSearchCV(cv=2, estimator=SVC(),
             param_grid={'C': [1, 10, 100], 'gamma': [0.01, 0.1, 1],
                         'kernel': ['linear', 'sigmoid']},
             verbose=3)

In [20]:
pred = clf.predict(test_text)
print('SVC accuracy: ', accuracy_score(pred, test_labels))

SVC accuracy:  0.8523333333333334


In [21]:
param_grid = {'solver':['liblinear','saga'], 'penalty': ['l1', 'l2']}
model = LogisticRegression()
clf = GridSearchCV(model, param_grid, cv=2, verbose=3)
clf.fit(train_text, train_labels)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV 1/2] END ......penalty=l1, solver=liblinear;, score=0.825 total time=   0.8s
[CV 2/2] END ......penalty=l1, solver=liblinear;, score=0.812 total time=   0.9s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


[CV 1/2] END ...........penalty=l1, solver=saga;, score=0.821 total time= 1.0min


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


[CV 2/2] END ...........penalty=l1, solver=saga;, score=0.814 total time= 1.0min
[CV 1/2] END ......penalty=l2, solver=liblinear;, score=0.830 total time=   0.7s
[CV 2/2] END ......penalty=l2, solver=liblinear;, score=0.825 total time=   0.7s
[CV 1/2] END ...........penalty=l2, solver=saga;, score=0.832 total time=   9.9s
[CV 2/2] END ...........penalty=l2, solver=saga;, score=0.827 total time=   9.1s


GridSearchCV(cv=2, estimator=LogisticRegression(),
             param_grid={'penalty': ['l1', 'l2'],
                         'solver': ['liblinear', 'saga']},
             verbose=3)

In [22]:
pred = clf.predict(test_text)
print('Logistic Regression accuracy: ', accuracy_score(pred, test_labels))

Logistic Regression accuracy:  0.8526666666666667
